## 2- Data Cleaning

In [1]:
# Essentials
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline

# Importing feature engineering library
from feature_engine import categorical_encoders as ce

In [2]:
# Loading raw data
df = pd.read_csv('Apartments_raw.csv', index_col='Unnamed: 0')
df.head()

,url,address,neighborhood,rent,beds,baths
0,https://www.renthop.com/listings/382-wadsworth...,"382 Wadsworth Ave, Apt 4G","Fort George, Washington Heights, Upper Manhatt...","$2,300",2_Bed,1_Bath
1,https://www.renthop.com/listings/162-east-33rd...,"162 East 33rd Street, Apt 3TW","Rose Hill, Kips Bay, Midtown Manhattan, Manhattan","$3,500",2_Bed,1_Bath
2,https://www.renthop.com/listings/400-east-71st...,"400 East 71st Street, Apt 12DE...","Upper East Side, Upper Manhattan, Manhattan","$5,795",3_Bed,2_Bath
3,https://www.renthop.com/listings/814-10th-aven...,"814 10th Avenue, Apt 7C","Hell's Kitchen, Midtown Manhattan, Manhattan","$2,225",Studio,1_Bath
4,https://www.renthop.com/listings/100-west-31st...,"100 West 31st Street, Apt 36G","Chelsea, Midtown Manhattan, Manhattan","$5,030",1_Bed,1_Bath


In [3]:
# Checking for missing values
df.isnull().mean()

url             0.0000
address         0.0005
neighborhood    0.0000
rent            0.0000
beds            0.0000
baths           0.0000
dtype: float64

In [4]:
# Dropping columns with missing adress
df.dropna(axis=0, inplace=True)

In [5]:
# Checking the unique values of beds and baths columns
print(df['beds'].unique())
print(df['baths'].unique())

['2_Bed' '3_Bed' 'Studio' '1_Bed' 'Loft' '4_Bed' '5_Bed' 'Room' '_2_Bed'
 '7_Bed' '_1_Bed' '_Studio' '6_Bed' '_4_Bed' '_3_Bed']
['1_Bath' '2_Bath' '3_Bath' '5_Bath' '_1_Bath' '1.5_Bath' '4_Bath'
 '_2_Bath' '2.5_Bath' '3.5_Bath' '7_Bath' '4.5_Bath' '5.5_Bath' '_3_Bath'
 '6_Bath']


In [6]:
# Correcting some of the values
df['beds'] = df['beds'].map(lambda x: x[1:] if x.startswith('_') else x) 
df['baths'] = df['baths'].map(lambda x: x[1:] if x.startswith('_') else x)

In [7]:
# Checking the result
print(df['beds'].unique())
print(df['baths'].unique())

['2_Bed' '3_Bed' 'Studio' '1_Bed' 'Loft' '4_Bed' '5_Bed' 'Room' '7_Bed'
 '6_Bed']
['1_Bath' '2_Bath' '3_Bath' '5_Bath' '1.5_Bath' '4_Bath' '2.5_Bath'
 '3.5_Bath' '7_Bath' '4.5_Bath' '5.5_Bath' '6_Bath']


In [8]:
# Dropping duplicate rows
df.drop_duplicates(inplace=True)

In [9]:
# Looking at some descriptive statistics to better understand the data
df.describe()

,url,address,neighborhood,rent,beds,baths
count,9443,9443,9443,9443,9443,9443
unique,9436,6255,210,1834,10,12
top,https://www.renthop.com/listings/71-east-110th...,Gold Street,"Financial District, Downtown Manhattan, Manhattan","$2,500",1_Bed,1_Bath
freq,2,62,775,124,3706,7309


In [10]:
# Performing some transformations
df['rent'] = df['rent'].map(lambda x: str(x).replace('$','').replace(',','')).astype('int') 
df['beds'] = df['beds'].map(lambda x: x.replace('_Bed', '')) 
df['beds'] = df['beds'].map(lambda x: x.replace('Studio', '0')) 
df['beds'] = df['beds'].map(lambda x: x.replace('Room', '0'))
df['beds'] = df['beds'].map(lambda x: x.replace('Loft', '0')).astype('int') 
df['baths'] = df['baths'].map(lambda x: x.replace('_Bath', '')).astype('float')

In [11]:
# Checking the new dtypes
df.dtypes

url              object
address          object
neighborhood     object
rent              int64
beds              int64
baths           float64
dtype: object

In [12]:
# Checking the number of apartment offers by neighborhood
df.groupby('neighborhood')['rent'].count().to_frame('count').sort_values(by='count', ascending=False)

,count
neighborhood,
"Financial District, Downtown Manhattan, Manhattan",775
"Hell's Kitchen, Midtown Manhattan, Manhattan",705
"Upper East Side, Upper Manhattan, Manhattan",542
"Yorkville, Upper East Side, Upper Manhattan, Manhattan",472
"Murray Hill, Midtown Manhattan, Manhattan",370
...,...
"Kingsbridge, Bronx",1
"Journal Square, Jersey City",1
"Hunters Point, Long Island City, Northwestern Queens, Queens",1


In [13]:
# Cleaning the neighborhood strings
df['neighborhood'] = df['neighborhood'].map(lambda x: x.strip())

In [14]:
# Checking the difference
df.groupby('neighborhood')['rent'].count().to_frame('count').sort_values(by='count', ascending=False)

,count
neighborhood,
"Financial District, Downtown Manhattan, Manhattan",779
"Hell's Kitchen, Midtown Manhattan, Manhattan",706
"Upper East Side, Upper Manhattan, Manhattan",543
"Yorkville, Upper East Side, Upper Manhattan, Manhattan",475
"Murray Hill, Midtown Manhattan, Manhattan",373
...,...
"Great Neck Plaza, Great Neck, North Hempstead",1
"Blissville, Northwestern Queens, Queens",1
"Mount Hope, West Bronx, Bronx",1


In [15]:
# Grouping neighborhoods with low frequency into "Rare" category
encoder = ce.RareLabelCategoricalEncoder(tol=0.005, variables=['neighborhood'])
encoder.fit(df)
df_t = encoder.transform(df)

In [16]:
# Dropping rows belonging to "Rare" category 
df_t = df_t[df_t['neighborhood'] != 'Rare'].reset_index(drop=True)
# Checking how many unique neighborhoods are left
print(len(df_t['neighborhood'].unique()))

43


In [17]:
# Dropping high cardinality variables
df_t.drop(['url', 'address'], axis=1,inplace=True)

In [18]:
# Checking the value counts
print(df_t.beds.value_counts())
print(df_t.baths.value_counts())

1    3241
2    2184
0    1643
3     805
4     235
5      30
6       9
7       3
Name: beds, dtype: int64
1.0    6291
2.0    1567
3.0     171
1.5      44
4.0      26
2.5      23
5.0      16
3.5       6
4.5       2
7.0       2
6.0       1
5.5       1
Name: baths, dtype: int64


In [19]:
# Grouping low frequency values together
df_t['beds'] = df_t['beds'].map(lambda x: x if x < 4 else 4)
df_t['baths'] = df_t['baths'].map(lambda x: x if x <= 3 else 4)

In [20]:
# Checking the cleaned dataframe
df_t.head()

,neighborhood,rent,beds,baths
0,"Rose Hill, Kips Bay, Midtown Manhattan, Manhattan",3500,2,1.0
1,"Upper East Side, Upper Manhattan, Manhattan",5795,3,2.0
2,"Hell's Kitchen, Midtown Manhattan, Manhattan",2225,0,1.0
3,"Chelsea, Midtown Manhattan, Manhattan",5030,1,1.0
4,"Hunters Point, Long Island City, Northwestern ...",3559,1,1.0


In [21]:
# Saving the cleaned dataframe for modeling
df_t.to_csv('Apartements_clean.csv')